In [1]:
from utils_project import *
from utils_timeseries import *
from info import task

2023-04-04 16:30:00.560995: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 60 timeseries files
Found 90 pca files
Found 90 iso files
Found 90 mds files
Found 90 lle files
Found 90 spec files
Found 90 umap files
Found 53 mask nifties


In [41]:
main_dir = f'{base_dir}/Trajectory_analyses/timeseries'
emb_dir = f'{main_dir}/embeddings'
embs = glob.glob(os.path.join(emb_dir, '*.pkl'))
print(f'Found {len(embs)} total embeddings')

# what to plot
rois_ = ['HPC-CA_thr25', 'HPC-DG_thr25', 'HPC-sub_thr25',
        'HPC-ant_thr25', 'HPC-mid_thr25', 'HPC-post_thr25', 'HPC_thr25', 
        'M1-a_thr25', 'M1-p_thr25','V1_thr25', 'postcentral_thr25',
        'ERC_thr25', 'amyg_thr25', 'mpfc_thr25']
rois = [f'L_{r}' for r in rois_] + [f'R_{r}' for r in rois_]
algos = ['lle' , 'spec', 'isomap'] 
nns = ['20', '50nn', '100nn']

plot_dir = f'{main_dir}/plots'
os.makedirs(plot_dir, exist_ok=True)

for algo in algos:

    # load in subjects & filter out exclusions
    algo_embs = [e for e in embs if algo in e]
    algo_embs = [e for e in algo_embs if int(e.split('/')[-1].split('_')[0]) in incl]
    n_subs = len(algo_embs)
    print(f'Found {n_subs} {algo} embeddings')

    per_row = 4
    n_rows = int(np.ceil(n_subs / per_row))
    fig = plt.figure(figsize=(20, 5 * n_rows))

    # for each roi & nearest neighbor number
    for combo in list(itertools.product(nns, rois)):
        nn, roi = combo
        out_fname = f'{plot_dir}/Decision-embedding_{algo}_{roi}_3D-{nn}nn_n{n_subs}.pdf'
        if os.path.exists(out_fname):
            print(f'Already exists: {out_fname}')
            continue

        # loop over subjects & plot
        for i, algo_emb in enumerate(algo_embs):

            sub_id = algo_emb.split('/')[-1].split('_')[0]
            emb_dict = pd.read_pickle(algo_emb)
            
            try: 

                emb = emb_dict[roi]['3d'][f'{nn}nn']['embedding']
                ax = fig.add_subplot(n_rows, per_row, i+1, projection='3d')
                plot_trajectory_with_plane(emb, title=f'P {sub_id}', ax=ax)

            except:

                ax = fig.add_subplot(n_rows, per_row, i+1)
                ax.text(0.5, 0.5, f'P {sub_id} MISSING', fontsize=16, ha='center', va='center')
                ax.axis('off')

        save_figures_pdf([fig], out_fname)

Found 2 total embeddings
Found 2 lle embeddings
Already exists: /Users/matty_gee/Desktop/plots/Decision-embedding_lle_L_HPC_thr25_3D-20nn_n2.pdf


<Figure size 2000x500 with 0 Axes>